## Convert the R1CS into a QAP


## Over Real Number

In [213]:
import numpy as np
import random

# Define the matrices
L = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick random values for x and y
x = random.randint(1,1000)
y = random.randint(1,1000)

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

print("witness {}".format(w))

result = O.dot(w) == np.multiply(L.dot(w),R.dot(w))
assert result.all(), "result contains an inequality"

witness [  1 301   2  15  12 180]


In [214]:
from scipy.interpolate import lagrange
import numpy as np
from numpy.polynomial import polynomial as P

np.set_printoptions(precision=2)


def poly_interpolate(index, matrics):
    res = []
    for row in matrics:
        coef = lagrange(index, row).coef[::-1]
        res.append(coef)
    return np.array(res, dtype=object)


def polymuln(lst):
    if len(lst) == 0:
        return 1
    return P.polymul(lst[0], polymuln(lst[1:]))


def QAP(s, L, R, O):
    U = np.matmul(s, L)
    V = np.matmul(s, R)
    W = np.matmul(s, O)
    print(f"L: {L}")
    print(f"R: {R}")
    print(f"O: {O}")
    Q = P.polysub(P.polymul(U, V), W)
    print(f"T:\n {Q}")
    assert len(U) == len(V) == len(W)
    return P.polydiv(Q, Z(len(U)))


def Z(n):
    ns = [[-(1 + i), 1] for i, _ in enumerate(range(n))]
    return polymuln(ns)


def r1cs_to_qap_real(s, L, R, O):
    x = np.arange(1, len(L) + 1)

    resL = poly_interpolate(x, np.transpose(L))
    resR = poly_interpolate(x, np.transpose(R))
    resO = poly_interpolate(x, np.transpose(O))

    print(f"interporated L: \n {resL}")
    print(f"interporated R: \n {resR}")
    print(f"interporated O: \n {resO}")

    return QAP(s, resL, resR, resO)



print(r1cs_to_qap_real(w, L, R, O))

interporated L: 
 [array([0.]) array([0.]) array([10., -9.,  2.]) array([0.])
 array([-3.,  4., -1.]) array([0.])]
interporated R: 
 [array([0.]) array([0.]) array([ 3. , -2.5,  0.5])
 array([ 2. , -3.5,  1.5]) array([0.]) array([0.])]
interporated O: 
 [[-3.0 4.5 -1.5]
 [1.0 -1.5 0.5]
 [1.0 -1.5 0.5]
 [2.0 -3.0 1.0]
 [3.0 -2.5 0.5]
 [-4.0 5.5 -1.5]]
L: [array([0.]) array([0.]) array([10., -9.,  2.]) array([0.])
 array([-3.,  4., -1.]) array([0.])]
R: [array([0.]) array([0.]) array([ 3. , -2.5,  0.5])
 array([ 2. , -3.5,  1.5]) array([0.]) array([0.])]
O: [[-3.0 4.5 -1.5]
 [1.0 -1.5 0.5]
 [1.0 -1.5 0.5]
 [2.0 -3.0 1.0]
 [3.0 -2.5 0.5]
 [-4.0 5.5 -1.5]]
T:
 [-222.0 1535.0 -2290.0 1165.0 -188.0]
(array([37.0, -188.0], dtype=object), array([0.0], dtype=object))


## Over Finite Field

In [215]:
import numpy as np
import random
import galois
# from py_ecc.bn128 import G1, multiply, add, curve_order, eq, Z1
from py_ecc.optimized_bn128 import curve_order, G1, G2, add, pairing, neg, normalize


# P = 71
# P = 21888242871839275222246405745257275088548364400416034343698204186575808495617
P = curve_order
GF = galois.GF(P)

L_F = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R_F = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O_F = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [GF(P-3),1,1,2,0,GF(P-1)]])

L_galois = GF(L_F)
R_galois = GF(R_F)
O_galois = GF(O_F)

# pick random values for x and y
x = GF(1)
y = GF(2)


# this is our original formula
v1 = 3*x*x
v2 = v1 * y
out = 3*x*x*y + 5*x*y + GF(P-1)*x + GF(P-2)*y + GF(3)
# the witness vector with the intermediate variables inside
w = GF(np.array([1, out, x, y, v1, v2]))

print("witness {}".format(w))

result = O_galois.dot(w) == np.multiply(L_galois.dot(w),R_galois.dot(w))
assert result.all(), "result contains an inequality"

witness [ 1 14  1  2  3  6]


In [216]:
from scipy.interpolate import lagrange
import numpy as np
import functools 
from numpy.polynomial import polynomial as P
from py_ecc.optimized_bn128 import curve_order, G1, G2, add, pairing, neg, normalize
import galois

# P = 71
# P = 21888242871839275222246405745257275088548364400416034343698204186575808495617
P = curve_order
GF = galois.GF(P)

np.set_printoptions(precision=2)


def poly_finite_field_interpolate(index, matrics):
    res = []
    for row in matrics:
        poly = galois.lagrange_poly(index, row)
        coef = poly.coefficients()[::-1]
        if len(coef) < matrics.shape[1]:
            coef = np.append(coef, np.zeros(matrics.shape[1] - len(coef), dtype=int))
        res.append(coef)
    return GF(res)


def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return functools.reduce(sum_, map(mul_, polys, witness))


def QAP(s, L, R, O):

    print(f"len(L): \n {len(L)}")

    U_poly = galois.Poly((s @ L)[::-1])
    V_poly = galois.Poly((s @ R)[::-1])
    W_poly = galois.Poly((s @ O)[::-1])

    print(f"U: {U_poly}")
    print(f"V: {V_poly}")
    print(f"W: {W_poly}")

    print(f"L shape: {L.shape}")
    print(f"L : {L}")

    T_poly = galois.Poly([1, P-1], field=GF)
    for i in range(2, L.shape[1] + 1):
        print(f"T X- : {i}")
        T_poly *= galois.Poly([1, P-i], field=GF)

    print("\nT = ", T_poly)

    H_poly = (U_poly * V_poly - W_poly) // T_poly
    print("\nH = ", H_poly)
    rem = (U_poly * V_poly - W_poly) % T_poly
    print("\nrem = ", rem)
    print("\nh(x)t(x = ", H_poly*T_poly)

    assert rem == 0
    assert U_poly * V_poly == W_poly + H_poly*T_poly, "division has a remainder"
    assert len(U) == len(V) == len(W)
    return U_poly, V_poly, W_poly, H_poly, T_poly


def r1cs_to_qap_field(s, L, R, O):
    x = GF(np.arange(1, len(L) + 1))

    print(f"len(L): \n {len(L)}")

    L_galois = GF(L)
    R_galois = GF(R)
    O_galois = GF(O)

    resL = poly_finite_field_interpolate(x, np.transpose(L_galois))
    resR = poly_finite_field_interpolate(x, np.transpose(R_galois))
    resO = poly_finite_field_interpolate(x, np.transpose(O_galois))

    print(f"interporated L: \n {resL}")
    print(f"interporated R: \n {resR}")
    print(f"interporated O: \n {resO}")

    return QAP(s, resL, resR, resO)


In [217]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

U, V, W, H, T = r1cs_to_qap_field(w, L_F, R_F, O_F)

print("\nU = ",U)
print("\nV = ",V)
print("\nW = ",W)
print("\nH = ",H)
print("\nT = ",T)

print("Setup phase")
print("-"*10)
print("Toxic waste:")

tau = GF(20)
print(f"τ = {tau}")


for i in range(1, T.degree + 2):
    print(f"T({i}) = ", T(i))
    if i == T.degree:
        print("-"*10)

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

assert u * v - _w == ht, f"{u} * {v} - {_w} != {ht}"


len(L): 
 3
interporated L: 
 [[                                                                            0
                                                                              0
                                                                              0]
 [                                                                            0
                                                                              0
                                                                              0]
 [                                                                           10
  21888242871839275222246405745257275088548364400416034343698204186575808495608
                                                                              2]
 [                                                                            0
                                                                              0
                                                                              0]
 [2188

In [218]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]
# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[T(τ)]G1 = {[normalize(point) for point in target_G1]}")

print(f"\n[τ]G2 = {[normalize(point) for point in tau_G2]}")

Trusted setup:
----------
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714)]
[T(τ)]G1 = [(15079658896578777727957253552140006753386033884163276984652000792054067003845, 15571690380226172156253627271507303654138844659817334859838817906959013556375), (2222527164631897436007971325910993322691342916064499345333731411094803510296, 14772488293582439904888846000299461680393227290277198974272133839314065070050)]

[τ]G2 = [((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 40823678758634336813322034031454355683168513275934012081

In [219]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize


def evaluate_poly(poly, trusted_points):
    coeff = poly.coefficients()[::-1]

    print(f"coefficients: {coeff}")

    assert len(coeff) == len(trusted_points), "Polynomial degree mismatch!"

    terms = [multiply(point, int(coeff)) for point, coeff in zip(trusted_points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    return evaluation

print("\nProof generation:")
print("-"*10)
# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
Cw_G1 = evaluate_poly(W, tau_G1)
# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)

C_G1 = add(Cw_G1, HT_G1)

print(f"normalized G1 = {normalize(G1)}")
print(f"normalized G2 = {normalize(G2)}")


print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")


print("\nProof verification:")
print("-"*10)
# e(A, B) == e(C, G2[1])
assert pairing(B_G2, A_G1) == pairing(G2, C_G1), "Pairing check failed!"
print("Pairing check passed!")


Proof generation:
----------
coefficients: [                                                                            1
                                                                             3
 21888242871839275222246405745257275088548364400416034343698204186575808495616]
coefficients: [                                                                            7
 10944121435919637611123202872628637544274182200208017171849102093287904247799
 10944121435919637611123202872628637544274182200208017171849102093287904247812]
coefficients: [                                                                            1
 10944121435919637611123202872628637544274182200208017171849102093287904247810
 10944121435919637611123202872628637544274182200208017171849102093287904247809]
coefficients: [21888242871839275222246405745257275088548364400416034343698204186575808495616
 10944121435919637611123202872628637544274182200208017171849102093287904247805]
normalized G1 = (1, 2)
normalized G2 = ((